In [ ]:
import open3d as o3d
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm   

In [ ]:
df = pd.read_pickle('result.pkl')

In [ ]:
total_frames = len(df)
lidar_bbox_data = {}
class_name = []
for i, frame in enumerate(df):
    lidar_bbox_data[f'{i:06d}'] = frame['boxes_lidar']
    class_name.append(frame['name'])

In [ ]:
point_cloud_data = []
for i in range(total_frames):
    # print(f'Processing frame {i}/{total_frames}')
    # pcd = o3d.io.read_point_cloud(f'/velodyne/{i:06d}.bin')
    pcd_data = np.load(f'velodyne_depth/{i:06d}.npy').reshape(-1, 4)
    np.asarray(pcd_data)
    # Reshape and drop reflection values
    # points = bin_pcd.reshape((-1, 4))[:, 0:3]
    pcd_data = pcd_data[:,:3]
    point_cloud_data.append(pcd_data)

In [ ]:
bbox_list = []
for i in range(total_frames):
    bboxes = []
    for bbox in lidar_bbox_data[f'{i:06d}']:
        l,w,h = bbox[3:6]
        position_bbox = bbox[:3]
        rotation_y = bbox[6]
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
        y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

        theta = -(np.pi/2 + rotation_y)
        R = np.array([[np.cos(theta), 0, np.sin(theta)],
                        [0, 1, 0],
                        [-np.sin(theta), 0, np.cos(theta)]])
        # R = tf3d.euler.euler2mat(0,0,rotation_y[i][j])
        corners3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)
        corners3d = np.dot(R, corners3d).T
        corners3d_lidar = corners3d + position_bbox
        bboxes.append(corners3d_lidar)
    bbox_list.append(bboxes)

In [ ]:
for i in range(len(bbox_list)):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud_data[i])
    pcd.paint_uniform_color([0, 0.5,0.5])
    geometries = [pcd]
    for bbox in bbox_list[i]:
        pcd2 = o3d.geometry.PointCloud()
        pcd2.points = o3d.utility.Vector3dVector(bbox)
        pcd2.paint_uniform_color([0, 0, 1])
        bb_data = o3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd2.points)
        obb_data = o3d.geometry.AxisAlignedBoundingBox.get_oriented_bounding_box(bb_data)
        geometries.append(obb_data)
    
    o3d.visualization.draw_geometries(geometries)

alternate visualization method


In [ ]:
pc_bin = np.fromfile('velodyne/0000000000.bin', dtype=np.float32).reshape(-1, 4)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_bin[:, :3])
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add the point cloud to the visualizer
vis.add_geometry(pcd)

# Add coordinate axes with a length of 1.0
vis.add_geometry(o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0))

# Customize the point size
opt = vis.get_render_option()
opt.point_size = 1.0  # Increase point size
opt.background_color = np.asarray([0, 0, 0])  # Set background to black

# Run the visualizer to display the point cloud
vis.run()

# Close the visualizer
vis.destroy_window()